In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import optimizers, layers
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

In [5]:
rain = pd.read_csv("../Data/Rainfall.csv")

In [6]:
import glob

slp_path = "../Data/Anomaly/slp/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)
  
slp_months.sort()
print(slp_months)

['../Data/Anomaly/slp\\slp1.csv', '../Data/Anomaly/slp\\slp10.csv', '../Data/Anomaly/slp\\slp11.csv', '../Data/Anomaly/slp\\slp12.csv', '../Data/Anomaly/slp\\slp2.csv', '../Data/Anomaly/slp\\slp3.csv', '../Data/Anomaly/slp\\slp4.csv', '../Data/Anomaly/slp\\slp5.csv', '../Data/Anomaly/slp\\slp6.csv', '../Data/Anomaly/slp\\slp7.csv', '../Data/Anomaly/slp\\slp8.csv', '../Data/Anomaly/slp\\slp9.csv']


In [7]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

878.0689655172414


In [8]:
from scipy.stats import pearsonr
correlation = []

# chacking for each MONTH
month_path = slp_months[1] #0th in dex means january
# reading month data
month_data = pd.read_csv(month_path)
# month_data.head()
# len(month_data)

In [9]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

combine = []
for i in range(12):
  month_path = slp_months[i]
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]
  for i in feature:
    combine.append(i)
x = np.array(combine)
x.shape

(636, 324)

In [10]:
def get_lr(initial_learning_rate=0.001, decay_steps=4000, decay_rate=1):
    return optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=decay_steps,
            decay_rate=decay_rate,
            staircase=True)

In [11]:
def get_model_1(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(97)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [12]:
model_1 = get_model_1()
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense (Dense)               (None, 324)               105300    
                                                                 
 re_lu (ReLU)                (None, 324)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                31525     
                                                                 
 dense_2 (Dense)             (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/800
40/40 - 0s - loss: 11.6885 - val_loss: 7.5564 - 390ms/epoch - 10ms/step
Epoch 2/800
40/40 - 0s - loss: 6.2635 - val_loss: 4.9157 - 63ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 4.3367 - val_loss: 3.5797 - 64ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 3.3021 - val_loss: 2.8043 - 64ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 2.6838 - val_loss: 2.3416 - 63ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 2.2796 - val_loss: 2.0316 - 65ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 1.9965 - val_loss: 1.7877 - 64ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 1.7878 - val_loss: 1.6104 - 65ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 1.6235 - val_loss: 1.4767 - 65ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 1.4918 - val_loss: 1.3588 - 65ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 1.3792 - val_loss: 1.2552 - 64ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - loss: 1.2811 - val_loss: 1.1837 - 63ms/epoch - 2ms/step
Epoch 13/800
40/40 - 0

In [14]:
model_1.layers[3].get_weights()[0].shape

(324, 97)

In [15]:
model_temp1 = Model(inputs=model_1.input, outputs=model_1.layers[3].output)
model_2_inputs = model_temp1(x)
model_2_inputs.shape

TensorShape([636, 97])

In [16]:
# define encoder
def get_model_2(n_inputs=97):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(29)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [17]:
model_2 = get_model_2()
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 97)]              0         
                                                                 
 dense_3 (Dense)             (None, 97)                9506      
                                                                 
 re_lu_1 (ReLU)              (None, 97)                0         
                                                                 
 dense_4 (Dense)             (None, 29)                2842      
                                                                 
 dense_5 (Dense)             (None, 97)                2910      
                                                                 
Total params: 15,258
Trainable params: 15,258
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model_2.fit(model_2_inputs, model_2_inputs, epochs=800, batch_size=16, verbose=2, validation_data=(model_2_inputs,model_2_inputs))

Epoch 1/800
40/40 - 0s - loss: 23.1972 - val_loss: 19.8408 - 310ms/epoch - 8ms/step
Epoch 2/800
40/40 - 0s - loss: 18.3347 - val_loss: 16.5183 - 53ms/epoch - 1ms/step
Epoch 3/800
40/40 - 0s - loss: 15.1778 - val_loss: 13.6494 - 51ms/epoch - 1ms/step
Epoch 4/800
40/40 - 0s - loss: 12.9223 - val_loss: 11.9383 - 53ms/epoch - 1ms/step
Epoch 5/800
40/40 - 0s - loss: 11.4677 - val_loss: 10.7461 - 54ms/epoch - 1ms/step
Epoch 6/800
40/40 - 0s - loss: 10.4240 - val_loss: 9.8223 - 53ms/epoch - 1ms/step
Epoch 7/800
40/40 - 0s - loss: 9.5842 - val_loss: 9.0677 - 52ms/epoch - 1ms/step
Epoch 8/800
40/40 - 0s - loss: 8.8956 - val_loss: 8.4702 - 52ms/epoch - 1ms/step
Epoch 9/800
40/40 - 0s - loss: 8.3440 - val_loss: 7.9837 - 51ms/epoch - 1ms/step
Epoch 10/800
40/40 - 0s - loss: 7.9138 - val_loss: 7.5937 - 53ms/epoch - 1ms/step
Epoch 11/800
40/40 - 0s - loss: 7.5566 - val_loss: 7.2806 - 52ms/epoch - 1ms/step
Epoch 12/800
40/40 - 0s - loss: 7.2755 - val_loss: 7.0294 - 53ms/epoch - 1ms/step
Epoch 13/800


In [19]:
model_temp2 = Model(inputs=model_2.input, outputs=model_2.layers[3].output)
model_3_inputs = model_temp2(model_2_inputs)
model_3_inputs.shape

TensorShape([636, 29])

In [20]:
def get_model_3(n_inputs=29):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
  return model

In [21]:
model_3 = get_model_3()
model_3.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 29)]              0         
                                                                 
 dense_6 (Dense)             (None, 29)                870       
                                                                 
 re_lu_2 (ReLU)              (None, 29)                0         
                                                                 
 dense_7 (Dense)             (None, 9)                 270       
                                                                 
 dense_8 (Dense)             (None, 29)                290       
                                                                 
Total params: 1,430
Trainable params: 1,430
Non-trainable params: 0
_________________________________________________________________


In [22]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(model_3_inputs, model_3_inputs, epochs=800, batch_size=16, verbose=2, validation_data=(model_3_inputs,model_3_inputs))
# plot loss

Epoch 1/800
40/40 - 0s - loss: 17.9984 - val_loss: 15.9430 - 310ms/epoch - 8ms/step
Epoch 2/800
40/40 - 0s - loss: 15.0883 - val_loss: 14.3218 - 51ms/epoch - 1ms/step
Epoch 3/800
40/40 - 0s - loss: 13.8588 - val_loss: 13.3801 - 51ms/epoch - 1ms/step
Epoch 4/800
40/40 - 0s - loss: 13.0288 - val_loss: 12.6398 - 51ms/epoch - 1ms/step
Epoch 5/800
40/40 - 0s - loss: 12.3557 - val_loss: 12.0249 - 51ms/epoch - 1ms/step
Epoch 6/800
40/40 - 0s - loss: 11.7902 - val_loss: 11.4990 - 52ms/epoch - 1ms/step
Epoch 7/800
40/40 - 0s - loss: 11.3128 - val_loss: 11.0654 - 51ms/epoch - 1ms/step
Epoch 8/800
40/40 - 0s - loss: 10.9205 - val_loss: 10.7162 - 51ms/epoch - 1ms/step
Epoch 9/800
40/40 - 0s - loss: 10.6023 - val_loss: 10.4342 - 52ms/epoch - 1ms/step
Epoch 10/800
40/40 - 0s - loss: 10.3472 - val_loss: 10.2070 - 51ms/epoch - 1ms/step
Epoch 11/800
40/40 - 0s - loss: 10.1438 - val_loss: 10.0284 - 50ms/epoch - 1ms/step
Epoch 12/800
40/40 - 0s - loss: 9.9814 - val_loss: 9.8750 - 54ms/epoch - 1ms/step
Ep

In [23]:
def get_model_final(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  e = Dense(29)(e)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  e = Dense(29)(bottleneck)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  output = Dense(n_inputs, activation='linear')(e)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [24]:
model_final = get_model_final()
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_9 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_3 (ReLU)              (None, 324)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [25]:
model_final.layers[2] = model_1.layers[3]
model_final.layers[11] = model_1.layers[4]
model_final.layers[4] = model_2.layers[3]
model_final.layers[9] = model_2.layers[4]
model_final.layers[6] = model_3.layers[3]
model_final.layers[7] = model_3.layers[4]
model_final.compile(optimizer='adam', loss='mse')

In [26]:
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_9 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_3 (ReLU)              (None, 324)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [27]:
history = model_final.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))


Epoch 1/1000
40/40 - 0s - loss: 13.4083 - val_loss: 10.2567 - 440ms/epoch - 11ms/step
Epoch 2/1000
40/40 - 0s - loss: 9.6144 - val_loss: 8.9855 - 69ms/epoch - 2ms/step
Epoch 3/1000
40/40 - 0s - loss: 8.8275 - val_loss: 8.5433 - 70ms/epoch - 2ms/step
Epoch 4/1000
40/40 - 0s - loss: 8.4290 - val_loss: 8.0553 - 70ms/epoch - 2ms/step
Epoch 5/1000
40/40 - 0s - loss: 7.9183 - val_loss: 7.5463 - 69ms/epoch - 2ms/step
Epoch 6/1000
40/40 - 0s - loss: 7.3667 - val_loss: 7.0303 - 68ms/epoch - 2ms/step
Epoch 7/1000
40/40 - 0s - loss: 7.0199 - val_loss: 6.7232 - 70ms/epoch - 2ms/step
Epoch 8/1000
40/40 - 0s - loss: 6.6917 - val_loss: 6.4613 - 69ms/epoch - 2ms/step
Epoch 9/1000
40/40 - 0s - loss: 6.5147 - val_loss: 6.3384 - 71ms/epoch - 2ms/step
Epoch 10/1000
40/40 - 0s - loss: 6.4017 - val_loss: 6.2100 - 70ms/epoch - 2ms/step
Epoch 11/1000
40/40 - 0s - loss: 6.2120 - val_loss: 6.0574 - 68ms/epoch - 2ms/step
Epoch 12/1000
40/40 - 0s - loss: 6.0598 - val_loss: 5.9238 - 70ms/epoch - 2ms/step
Epoch 13/

In [28]:
# model_final.save("../Data/Saved Models/model_final_slp.h5")
# model_final = keras.models.load_model('./Data/Models/model_final.h5')

In [29]:
class Custom_Layer(layers.Layer):
    def __init__(self, weights, bias=False):
        super(Custom_Layer, self).__init__()
        self.w = weights
        if bias:
            self.b = bias

    def call(self, inputs):
        try: return tf.matmul(inputs, self.w) + self.b
        except: return tf.matmul(inputs, self.w)

In [30]:
std = []
for i in model_final.layers:
    # print(i.weights[0])
    try: std.append(np.array(math.reduce_std(i.weights[0])) * 1.9)
    except: std.append([])

In [31]:
final_layers = keras.Sequential([Input(shape=(None, 324))])
for i, layer in enumerate(model_final.layers):
    try:
        temp_layer = []
        for ind, j in enumerate(layer.weights[0]):
            temp_weights = []
            for index, k in enumerate(j):
                if k < std[i]: temp_weights.append(0)
                else:          temp_weights.append(1)  
            temp_layer.append(temp_weights)
        weights = tf.Variable(temp_layer, dtype='float32')
        try: final_layers.add(Custom_Layer(weights, bias=layer.weights[1]))
        except: final_layers.add(Custom_Layer(weights))
    except IndexError: final_layers.add(layer)

In [32]:
final_layers.compile(optimizer='adam', loss='mse')
final_layers.build(input_shape=(None, 324))
final_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        multiple                  0         
                                                                 
 custom__layer_1 (Custom_Lay  (None, None, 324)        104976    
 er)                                                             
                                                                 
 re_lu_3 (ReLU)              multiple                  0         
                                                                 
 custom__layer_3 (Custom_Lay  (None, None, 97)         31428     
 er)                                                             
                                                                 
 re_lu_4 (ReLU)              multiple                  0         
                                                                 
 custom__layer_5 (Custom_Lay  (None, None, 29)         2

In [33]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final3.trainable = False

models = [model_final1, model_final2, model_final3]
# models = [model_final1, model_final4]

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")


In [34]:
final_layers1 = Model(inputs=final_layers.input, outputs=final_layers.layers[3].output)
final_layers1.trainable = False

final_layers2 = Model(inputs=final_layers.input, outputs=final_layers.layers[5].output)
final_layers2.trainable = False

final_layers3 = Model(inputs=final_layers.input, outputs=final_layers.layers[7].output)
final_layers3.trainable = False

models = [final_layers1, final_layers2, final_layers3]

In [35]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,mod):
  # chacking for each MONTH
  month_path = slp_months[month] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:]  #Here i am using anly data from 1948 to 2000
  pred = np.array(feature)
  try: pred_m = models[mod](feature)
  except: 
    print(i)
    raise(IndexError)
  pred_f = pred_m.numpy()
  return pred_f

In [36]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in range(len(models)):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [37]:
from itertools import combinations
# months_comb = [j for i in range(4) for j in combinations(range(5,9), i+1)]
months_comb = [[4]]
top = 9
# models = [model_final1, model_final2, model_final3]
models = [final_layers1, final_layers2, final_layers3]

minimum, maximum = 1, -1
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print("\n\nMonth combination = ",month,"    top = ",i,"   min and max",left,"   ",right)
    if minimum > left:  minimum, min_month, min_top = left, month, i
    if maximum < right: maximum, max_month, max_top = right, month, i
    print(f'Minimum: {minimum}, {min_month}, {min_top}    Maximum: {maximum}, {max_month}, {max_top}\n')

Window size =  10    plcc -0.3019141794764296
Window size =  11    plcc -0.45105517005135737
Window size =  12    plcc -0.47039526168560253
Window size =  13    plcc -0.39850644613131475
Window size =  14    plcc -0.5084063407885724
Window size =  15    plcc -0.4352727908382794
Window size =  16    plcc -0.2799359212459818
Window size =  17    plcc -0.3979708765191239
Window size =  18    plcc -0.42400835303907697
Window size =  19    plcc -0.36600122518246625
Window size =  20    plcc -0.42428052483305734
Window size =  21    plcc -0.19899262184456354
Window size =  22    plcc -0.4072493915641778
Window size =  23    plcc -0.23643986575370443
Window size =  24    plcc -0.23279458918901247
Window size =  25    plcc -0.20159853235267985
Window size =  26    plcc -0.270716243711045
Window size =  27    plcc -0.25708207279422846
Window size =  28    plcc -0.27158560550821403
Window size =  29    plcc -0.30982828256478134
Window size =  30    plcc -0.2616459253383408
Window size =  31    p